In [9]:
!pip install topmost

In [10]:
import topmost
import os

In [11]:
device = "cuda"  # or "cpu"
dataset_dir = "/kaggle/input/revista-de-ciencias-mdicas-de-la-habana-cuba"
output_dir = "./results"
os.makedirs(output_dir, exist_ok=True)

dataset = topmost.data.DynamicDataset(dataset_dir, batch_size=200, read_labels=True, device=device)

train size:  1208
test size:  658
vocab size:  19384
average length: 1335.378
num of each time slice:  22 [20 34 46 41 56 20 76 58 43 54 83 71 64 67 63 60 55 83 75 58 57 24]


In [12]:
import random
import joblib
import json
import os

In [13]:
def save_txt(obj, path):
    with open(path, 'w') as file:
        file.write(obj)

def save_top_words_txt(top_words, path):
    with open(os.path.join(path, 'top_words.txt'), 'w') as file:
        for i, time in enumerate(top_words):
            file.write(f'================= Time {i} ================= \n')
            for j, topic in enumerate(time):
                file.write(f'================= Topic {j} ================= \n')
                file.write(f'{topic}')
                file.write('\n')
            file.write('\n')

def save_json(obj, path):
    with open(path, 'w') as file:
        json.dump(obj, file)

def save_joblib(obj, path):
    joblib.dump(obj, path)

def save_result(top_words, trainer, config, metrics_json):
    global samples 
    path = os.path.join(output_dir, f'{samples}')
    os.makedirs(path, exist_ok=True)

    try:
        save_top_words_txt(top_words, path)
        save_json(config, os.path.join(path, 'hyperparameters.json'))
        save_json(metrics_json, os.path.join(path, 'metrics_json.json'))
        save_json({"config": config, "metrics": metrics_json}, os.path.join(path, 'metrics_config.json'))
        # save_joblib(trainer, os.path.join(path, 'trainer.joblib'))
        save_joblib(top_words, os.path.join(path, 'top_words.joblib'))
        print('🍀 Saved')
    except Exception as e:
        traceback.print_exc(e)        

def get_hyperparameter_space():
    return {
  "training": {
    "learning_rate": [0.001, 0.002, 0.005, 0.01],
    "batch_size": [100, 200, 300, 500],
    "num_epoch": [400, 600, 800, 1000]
  },
  "model": {
    "num_topics": [20, 50, 100],
    "en1_units": [50, 100, 200, 300],
    "dropout": [0.0, 0.1, 0.3, 0.5],
    "beta_temp": [0.5, 1.0, 1.5, 2.0],
    "temperature": [0.05, 0.1, 0.2, 0.5],
    "weight_neg": [1e6, 5e6, 1e7, 1e8],
    "weight_pos": [1.0, 10.0, 100.0],
    "weight_UWE": [1e2, 1e3, 1e4],
    "neg_topk": [5, 10, 15, 20, 30]
  }
}

def get_default_config():
    return { "training": {
                "learning_rate": 0.002,
                "batch_size": 200,
                "num_epoch": 800 },

            "model": {
                "num_topics": 50,
                "en1_units": 100,
                "dropout": 0. , 
                "beta_temp": 1.0,
                "temperature": 0.1,
                "weight_neg": 1.0e+7,
                "weight_pos": 1.0e+1,
                "weight_UWE": 1.0e+3,
                "neg_topk": 15 }
           }

def random_configuration(hyperparameter_space):
    return { "training": {
                "learning_rate": random.choice(hyperparameter_space["training"]["learning_rate"]),
                "batch_size": random.choice(hyperparameter_space["training"]["batch_size"]),
                "num_epoch": random.choice(hyperparameter_space["training"]["num_epoch"]) },

            "model": {
                "num_topics": random.choice(hyperparameter_space["model"]["num_topics"]),
                "en1_units": random.choice(hyperparameter_space["model"]["en1_units"]),
                "dropout": random.choice(hyperparameter_space["model"]["dropout"]) , 
                "beta_temp": random.choice(hyperparameter_space["model"]["beta_temp"]),
                "temperature": random.choice(hyperparameter_space["model"]["temperature"]),
                "weight_neg": random.choice(hyperparameter_space["model"]["weight_neg"]),
                "weight_pos": random.choice(hyperparameter_space["model"]["weight_pos"]),
                "weight_UWE": random.choice(hyperparameter_space["model"]["weight_UWE"]),
                "neg_topk":random.choice(hyperparameter_space["model"]["neg_topk"]) }
           }

## Hyperparameters Search

## - Evaluacion

In [14]:
########################### Evaluate ####################################
import numpy as np
from topmost import eva
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel
from topmost.data.file_utils import split_text_word

In [15]:
def _coherence_modified(reference_corpus, vocab, top_words, cv_type='c_v'):
    # print('🍀')
    
    split_top_words = split_text_word(top_words)
    num_top_words = len(split_top_words[0])
    for item in split_top_words:
        assert num_top_words == len(item)

    split_reference_corpus = split_text_word(reference_corpus)
    dictionary = Dictionary(split_text_word(vocab))

    cm = CoherenceModel(texts=split_reference_corpus, dictionary=dictionary, topics=split_top_words, topn=num_top_words, coherence=cv_type)
    cv_per_topic = cm.get_coherence_per_topic()
    # print(f"Coherence scores per topic: {cv_per_topic}")

    valid_scores = [score for score in cv_per_topic if not np.isnan(score)]
    if not valid_scores:
        # raise ValueError("All coherence scores are NaN.")
        return 0
    score = np.mean(valid_scores)

    return score
    
eva.topic_coherence._coherence = _coherence_modified

In [14]:
def eval(trainer, top_words):
    # get theta (doc-topic distributions)
    train_theta, test_theta = trainer.export_theta()

    train_times = dataset.train_times.cpu().numpy()
    # compute topic coherence
    dynamic_TC = eva.dynamic_coherence(dataset.train_texts, train_times, dataset.vocab, top_words)
    print("dynamic_TC: ", dynamic_TC)

    # compute topic diversity
    dynamic_TD = eva.dynamic_diversity(top_words, dataset.train_bow.cpu().numpy(), train_times, dataset.vocab)
    print("dynamic_TD: ", dynamic_TD)

    # evaluate clustering
    clustering = eva._clustering(test_theta, dataset.test_labels)
    print(clustering)

    # evaluate classification
    classification = eva._cls(train_theta, test_theta, dataset.train_labels, dataset.test_labels)
    print(classification)

    json = {
        "dynamic_TC": dynamic_TC,
        "dynamic_TD": dynamic_TD,
        "clustering": clustering,
        "classification": classification
    }
    
    return json, dynamic_TC, dynamic_TD, clustering, classification

## - Modelo

In [10]:
import traceback

In [11]:
def evaluate_configuration(config):
    global tested_configs, samples
    
    config_key = json.dumps(config, sort_keys=True) 
    
    if config_key in tested_configs:
        return tested_configs[config_key]
    
    model = topmost.CFDTM(
        vocab_size=dataset.vocab_size,
        train_time_wordfreq=dataset.train_time_wordfreq,
        num_times=dataset.num_times,
        pretrained_WE=dataset.pretrained_WE,
        num_topics=config["model"]["num_topics"],
        en_units=config["model"]["en1_units"],
        temperature=config["model"]["temperature"],
        beta_temp=config["model"]["beta_temp"],
        weight_neg=config["model"]["weight_neg"],
        weight_pos=config["model"]["weight_pos"],
        weight_UWE=config["model"]["weight_UWE"],
        neg_topk=config["model"]["neg_topk"],
        dropout=config["model"]["dropout"],
        embed_size=300
    )
  
    model = model.to(device)  
    trainer = topmost.DynamicTrainer(model, dataset, batch_size=config["training"]["batch_size"], learning_rate=config["training"]["learning_rate"], epochs=config["training"]["num_epoch"])
    top_words, _ = trainer.train()
    
    try:
        metrics_json, dynamic_TC, dynamic_TD, clustering, classification = eval(top_words, trainer)
        #tested_configs[config_key] = 0.5 * dynamic_TC + 0.5 * dynamic_TD
        tested_configs[config_key] = {"dynamic_TC": dynamic_TC, "dynamic_TD": dynamic_TD}
        save_result(top_words, trainer, config, metrics_json)
        samples += 1
    except Exception as e:
        tested_configs[config_key] = {"dynamic_TC": float('-inf'), "dynamic_TD": float('-inf')}
        print(e)
        
    save_joblib(tested_configs, os.path.join(output_dir, 'tested_configs.joblib'))
    
    return tested_configs[config_key]

## - Algortimo evolutivo

In [11]:
def key_func(value):
    try:
        return 0.5 * value[1]["dynamic_TC"] + 0.5 * value[1]["dynamic_TD"]
    except: 
        return value[1]

def evolutionary_search(search_space, generations=10, mutation_rate=0.1, population_size=20, population=None, generation_count=0):
    global samples
    
    if not population:
        population = [random_configuration(search_space) for _ in range(population_size - 1)]
        population.append(get_default_config())

    # print(population)
    
    for generation in range(generation_count, generations):
        scores = [(config, evaluate_configuration(config)) for config in population]
        scores.sort(key=key_func, reverse=True)  

        print(f"🧬 Generación {generation + 1}, mejor resultado: {scores[0][1]}")
        
        num_parents = population_size // 2
        parents = [config for config, _ in scores[:num_parents]]
        
        children = []
        while len(children) < population_size - num_parents:
            parent_1, parent_2 = random.sample(parents, 2)
            child = {
                "model": {key: random.choice([parent_1["model"][key], parent_2["model"][key]]) for key in search_space["model"].keys()},
                "training": {key: random.choice([parent_1["training"][key], parent_2["training"][key]]) for key in search_space["training"].keys()},
            }
            
            if random.random() < mutation_rate:
                mutate_model_training = random.choice(list(search_space.keys()))
                print(mutate_model_training)
                param_to_mutate = random.choice(list(search_space[mutate_model_training].keys()))
                print(param_to_mutate)
                child[mutate_model_training][param_to_mutate] = random.choice(search_space[mutate_model_training][param_to_mutate])
            
            children.append(child)
        
        population = parents + children
        joblib.dump(population, os.path.join(output_dir, 'population.joblib'))
    
    best_config, best_score = max(scores, key=key_func)
    return best_config, best_score

## Main

In [13]:
population_size = 50
# population = None
population = joblib.load(os.path.join(output_dir, 'population.joblib'))

In [14]:
import joblib
# tested_configs = dict()
tested_configs = joblib.load('/kaggle/working/results/tested_configs.joblib')
# print(tested_configs)

In [15]:
results_path = '/kaggle/working/results'
results_path = [dir_.path for dir_ in os.scandir(results_path) if not dir_.is_file()]

print(len(results_path))

samples = len(results_path)

121


In [ ]:
hyperparameter_space = get_hyperparameter_space()

best_config, best_score = evolutionary_search(hyperparameter_space, population_size=population_size, generations=100, mutation_rate=0.2, population=population)
print(best_config)
print(best_score)

## Evaluando resultados

In [ ]:
results_path = '/kaggle/working/results'
results_path = [dir_.path for dir_ in os.scandir(results_path) if not dir_.is_file()]

print(len(results_path))

In [24]:
old_tc = float('-inf')
TC = []
old_td = float('-inf')
TD = []
old_tc_td = float('-inf')
TC_TD = []

for path in results_path:
    with open(os.path.join(path, 'metrics_json.json'), 'r') as file:
        metrics = json.load(file)

    cr_tc = metrics["dynamic_TC"]
    if cr_tc >= old_tc:
        with open(os.path.join(path, 'metrics_config.json'), 'r') as file:
            new = json.load(file)
        if cr_tc == old_tc:
            TC.append((new, path))
        else:
            old_tc = cr_tc
            TC = [(new, path)]

    cr_td = metrics["dynamic_TD"]
    if cr_td >= old_td:
        with open(os.path.join(path, 'metrics_config.json'), 'r') as file:
            new = json.load(file)
        if cr_td == old_td:
            TD.append((new, path))
        else:
            old_td = cr_td
            TD = [(new, path)]

    cr_tc_td = 0.5 * cr_tc + 0.5 * cr_td
    if cr_tc_td >= old_tc_td:
        with open(os.path.join(path, 'metrics_config.json'), 'r') as file:
            new = json.load(file)
        if cr_tc_td == old_tc_td:
            TC_TD.append((new, path))
        else:
            old_tc_td = cr_tc_td
            TC_TD = [(new, path)]

print(TC)
print(len(TC))
print('-------------------------------------')
print(TD)
print(len(TD))
print('-------------------------------------')
print(TC_TD)
print(len(TC_TD))

best_results_path = './best_results'
os.makedirs(best_results_path, exist_ok=True)
save_json(TC, os.path.join(best_results_path, 'best_tc.json'))
save_json(TD, os.path.join(best_results_path, 'best_td.json'))
save_json(TC_TD, os.path.join(best_results_path, 'best_tc_td.json'))

[({'config': {'training': {'learning_rate': 0.01, 'batch_size': 500, 'num_epoch': 1000}, 'model': {'num_topics': 100, 'en1_units': 50, 'dropout': 0.5, 'beta_temp': 1.5, 'temperature': 0.2, 'weight_neg': 5000000.0, 'weight_pos': 100.0, 'weight_UWE': 10000.0, 'neg_topk': 10}}, 'metrics': {'dynamic_TC': 0.6263097214101826, 'dynamic_TD': 0.335939393939394, 'clustering': {'Purity': 0.34498480243161095, 'NMI': 0.21334602154627538}, 'classification': {'acc': 0.3541033434650456, 'macro-F1': 0.12100456580356023}}}, '/kaggle/working/results/55')]
1
-------------------------------------
[({'config': {'model': {'num_topics': 20, 'en1_units': 50, 'dropout': 0.1, 'beta_temp': 1.5, 'temperature': 0.1, 'weight_neg': 5000000.0, 'weight_pos': 100.0, 'weight_UWE': 10000.0, 'neg_topk': 20}, 'training': {'learning_rate': 0.001, 'batch_size': 500, 'num_epoch': 800}}, 'metrics': {'dynamic_TC': 0.5281898315085433, 'dynamic_TD': 0.941060606060606, 'clustering': {'Purity': 0.3844984802431611, 'NMI': 0.273024515

In [ ]:
!zip -r best_results/best_results.zip /kaggle/working/results/84

In [ ]:
import joblib
top_words = joblib.load('/kaggle/working/results/84/top_words.joblib')
print(top_words)

## Prueba unica

In [ ]:
        config = {"model": {"num_topics": 20, "en1_units": 50, "dropout": 0.1, "beta_temp": 1.5, "temperature": 0.1, "weight_neg": 5000000.0, "weight_pos": 100.0, "weight_UWE": 10000.0, "neg_topk": 20}, "training": {"learning_rate": 0.001, "batch_size": 500, "num_epoch": 800}}

        model = topmost.CFDTM(
        vocab_size=dataset.vocab_size,
        train_time_wordfreq=dataset.train_time_wordfreq,
        num_times=dataset.num_times,
        pretrained_WE=dataset.pretrained_WE,
        num_topics=config["model"]["num_topics"],
        en_units=config["model"]["en1_units"],
        temperature=config["model"]["temperature"],
        beta_temp=config["model"]["beta_temp"],
        weight_neg=config["model"]["weight_neg"],
        weight_pos=config["model"]["weight_pos"],
        weight_UWE=config["model"]["weight_UWE"],
        neg_topk=config["model"]["neg_topk"],
        dropout=config["model"]["dropout"],
        embed_size=300
        )
  
        model = model.to(device)  
        trainer = topmost.DynamicTrainer(model, dataset, batch_size=config["training"]["batch_size"], learning_rate=config["training"]["learning_rate"], epochs=config["training"]["num_epoch"])
        top_words, _ = trainer.train()
        metrics_json, dynamic_TC, dynamic_TD, clustering, classification = eval(top_words, trainer)

 ## beta (================= es este)

In [ ]:
import joblib
#beta = model.get_beta()
#print(beta)
beta = joblib.load('top_words/beta.joblib')

#joblib.dump(beta, 'top_words/beta.joblib')

In [ ]:
print(beta.shape[0])
print(beta.shape[1])
print(beta.shape[2])

In [ ]:
import torch
def get_top_words(num=10):
    top_words_path = f'./top_words/{num}top_words_per_topics_model149'
    path_0 = os.path.join(top_words_path, 'separate_by_space')
    path_1 = os.path.join(top_words_path, 'separate_by_\n')
    # os.makedirs(top_words_path, exist_ok=True)
    os.makedirs(path_0, exist_ok=True)
    os.makedirs(path_1, exist_ok=True)

    top_words = []
    for topic in range(beta.shape[1]):
        # el range comienza en 9 para comenzar a partir del anno 2010
        top_words.append([[vocab[idx.item()] for idx in torch.topk(beta[time][topic], k=num).indices] for time in range(9, beta.shape[0])])
    # print(len(top_words))
    # print(len(top_words[0]))
    joblib.dump(top_words, f'{top_words_path}/{num}top_words_topic_149.joblib')

    print('Numero de topicos: ', len(top_words))
    print('Numero de annos: ', len(top_words[0]))

    
    for j, topic in enumerate(top_words):
        text = '' # 'Time 2003-2024 \n\n'
        text_1 = ''
        for i, time in enumerate(topic):
            text += f'{2012 + i}: ' + ' '.join(time) + '\n\n'
            text_1 += f'{2012 + i}\n' + '\n'.join(time) + '\n\n'
        with open(os.path.join(path_0, f'{j}.txt'), 'w') as file:
            file.write(text)
        with open(os.path.join(path_1, f'{j}.txt'), 'w') as file:
            file.write(text_1)

In [ ]:
!zip -r /kaggle/working/top_words/top_words_per_topics_model149.zip /kaggle/working/top_words

In [ ]:
import os
vocab_path = '/kaggle/input/revista-de-ciencias-mdicas-de-la-habana-cuba/vocab.txt'
with open(vocab_path, 'r') as file:
    words = file.readlines()
    vocab = [word.replace('\n', '') for word in words]

In [ ]:
for num in [10, 15, 20, 25, 30]:
    get_top_words(num)

In [ ]:
for num in [10, 15, 20, 25, 30]:
    os.remove(f'/kaggle/working/{num}top_words_topic_149.joblib')

## Visualizar heatmap

## Visualize (================= es este)

In [ ]:
import os
vocab_path = '/kaggle/input/revista-de-ciencias-mdicas-de-la-habana-cuba/vocab.txt'
with open(vocab_path, 'r') as file:
    words = file.readlines()
    vocab = [word.replace('\n', '') for word in words]

import joblib
beta = joblib.load('top_words/beta.joblib')

In [ ]:
!pip install unidecode

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def get_mapa_color(words, word_idxs, topic_name, path, annot_kws_size=8):
    data = dict()
    for time in range(9, 22):
        for word_idx in word_idxs:
            data[2003 + time] = data.get(2003 + time, []) + [beta[time][topic][word_idx].item()]

    df = pd.DataFrame(data, index=words)

    plt.figure(figsize=(10, 8))
    #sns.heatmap(df, cmap='Spectral', annot=True, fmt='d', cbar=True)
    sns.heatmap(df, cmap='Reds', annot=True, fmt='.2f', cbar=True, annot_kws={'size': annot_kws_size})
    #sns.heatmap(df, cmap='Blues', annot=True, fmt='d', cbar=True)
    #sns.heatmap(df, cmap='Reds', annot=True, fmt='d', cbar=True)

    # Leyenda
    plt.title(f"Mapa de Calor de Palabras por Año. {topic_name}", fontsize=16)
    plt.xlabel("Año", fontsize=10)
    plt.ylabel("Palabras Clave", fontsize=12)

    plt.savefig(path)
    plt.show()


In [ ]:
from unidecode import unidecode
images_path = './images'

def get_word_index(word):
    return vocab.index(unidecode(word)) 

### Topic 0

In [ ]:
topic = 0
topic_name = 'Tópico#0'
topic_path_img = f'{images_path}/topic_0/probabilidad.png'
os.makedirs(f'{images_path}/topic_0', exist_ok=True)
topic_path_img = f'{images_path}/topic_0/mapa_de_color.png'

words = ['laboral', 'uso', 'medio', 'nivel', 'permite', 'mala', 'abierto', 'cuestionario', 
             'confianza', 'burnout', 'anemia', 'acceso', 'comerciales', 'estudiantes', 'conocimiento', 
             'fatiga', 'digitales', 'ruido', 'videos']
word_index = [get_word_index(word) for word in words] 
get_mapa_color(words, word_index, topic_name, topic_path_img)

## Topico 1

In [ ]:
import os
topic = 1
topic_name = f'Tópico#{topic}'
os.makedirs(f'{images_path}/topic_{topic}', exist_ok=True)
# topic_path_img = f'{images_path}/topic_0/probabilidad.png'
topic_path_img = f'{images_path}/topic_{topic}/mapa_de_color.png'

words = [
    'historia', 'universidad', 'habana', 'medicina', 'cubana', 'cuba', 'cubanos', 'doctor', 'bloqueo', 'unidos', 'tropical', 'academia',
    'dorta', 'contreras', 'angiostrongylus', 'cantonensis', 'siglo', 'xix', 'carlos', 'instituto', 'universidad', 'hospital'
]

word_index = [get_word_index(word) for word in words]       
get_mapa_color(words, word_index, topic_name, topic_path_img)

# Topic 3

In [ ]:
import os
topic = 3
topic_name = f'Tópico#{topic}'
os.makedirs(f'{images_path}/topic_{topic}', exist_ok=True)
# topic_path_img = f'{images_path}/topic_0/probabilidad.png'
topic_path_img = f'{images_path}/topic_{topic}/mapa_de_color.png'

words = [
    'contenidos', 'habilidades', 'estudiante', 'plan', 'preguntas', 'carrera', 'instrumento', 'voz', 'lengua', 'sonidos', 'fuerza', 'english', 
    'sugerencias', 'clases', 'palabras', 'puntos', 'instrumento', 'ejercicio', 'preguntas', 'profesores', 'encuesta', 'cada', 'etapa', 
    'pruebas', 'entrenamiento', 'dificultad', 'recomendaciones', 'vacuna', 'vacunas', 'paso', 'producto','estrategia', 'etapa',
    'asignatura', 'basadas', 'examen'
]

get_mapa_color(words, word_index, topic_name, topic_path_img)

In [ ]:
import os
topic = 7
topic_name = f'Tópico#{topic}'
os.makedirs(f'{images_path}/topic_{topic}', exist_ok=True)
# topic_path_img = f'{images_path}/topic_0/probabilidad.png'
topic_path_img = f'{images_path}/topic_{topic}/mapa_de_color.png'

words = [
    'sida', 'vih', 'sexual', 'familiar', 'familia', 'vida', 'padres', 'mujeres', 'ancianos', 'envejecimiento', 'suicidio', 'relaciones', 'adolescentes',
    'personas', 'sexual', 'consumo', 'mental', 'estigma', 'conductas', 'cuidadores', 'trastorno', 'bipolar', 'intento', 'alcohol', 'demencia', 'estilos',
     'conducta', 'adultos'
]

get_mapa_color(words, word_index, topic_name, topic_path_img)

In [ ]:
import os
topic = 8
topic_name = f'Tópico#{topic}'
os.makedirs(f'{images_path}/topic_{topic}', exist_ok=True)
# topic_path_img = f'{images_path}/topic_0/probabilidad.png'
topic_path_img = f'{images_path}/topic_{topic}/mapa_de_color.png'

words = [ 'vigilancia', 'datos', 'casos', 'provincia', 'centros', 'fallecidos', 'red', 'disponible', 'muestras', 'fuente', 'intoxicaciones', 'ocurrencia',
         'consultas', 'agudos', 'fuente', 'alarma', 'dengue', 'lambayeque', 'pandemia', 'resistencia', 'impacto',
'cubanas', 'covid', 'competitividad', 'gobiernos', 'coronavirus', 'mortalidad', 'cov', 'sars', 'modelos', 'internet', 'dosis'
]

get_mapa_color(words, word_index, topic_name, topic_path_img)

In [ ]:
import os
topic = 9
topic_name = f'Tópico#{topic}'
os.makedirs(f'{images_path}/topic_{topic}', exist_ok=True)
# topic_path_img = f'{images_path}/topic_0/probabilidad.png'
topic_path_img = f'{images_path}/topic_{topic}/mapa_de_color.png'

words = [ 'hallazgos', 'lesiones', 'complicaciones', 'frecuentes', 'imagen', 'abdominal', 'fiebre', 'caso',  'quistes', 'intestinal', 'biopsia', 'frecuentes',
         'carcinoma', 'von']

get_mapa_color(words, word_index, topic_name, topic_path_img)

In [ ]:
import os
topic = 10
topic_name = f'Tópico#{topic}'
os.makedirs(f'{images_path}/topic_{topic}', exist_ok=True)
# topic_path_img = f'{images_path}/topic_0/probabilidad.png'
topic_path_img = f'{images_path}/topic_{topic}/mapa_de_color.png'

words = [
    'pylori', 'seguimiento', 'helicobacter', 'reportado', 'temporal',
'linfoma', 'manifestaciones', 'varicela', 'zoster', 'serial',
'gigantes', 'fetal', 'nervio', 'mano', 'ojo', 'síndrome', 'visual', 'seco', 
    'johnson', 'mediano', 'superficie', 'seguimiento', 'cuadro', 
    'aparece', 'entidad', 'consulta'
]

get_mapa_color(words, word_index, topic_name, topic_path_img)

In [ ]:
!zip -r /kaggle/working/images.zip /kaggle/working/images

## Defensa

## Modificacion: Número de tópicos = 14

In [16]:
defensa_path = './defensa'
os.makedirs(defensa_path, exist_ok=True)

def save_result_defensa(top_words, trainer, config, metrics_json, path):
    global samples 
    os.makedirs(path, exist_ok=True)

    try:
        save_top_words_txt(top_words, path)
        save_json(config, os.path.join(path, 'hyperparameters.json'))
        save_json(metrics_json, os.path.join(path, 'metrics_json.json'))
        save_json({"config": config, "metrics": metrics_json}, os.path.join(path, 'metrics_config.json'))
        #save_joblib(trainer, os.path.join(path, 'trainer.joblib'))
        save_joblib(top_words, os.path.join(path, 'top_words.joblib'))
    except Exception as e:
        traceback.print_exc()

In [ ]:
        config = {"model": {"num_topics": 14, "en1_units": 50, "dropout": 0.1, "beta_temp": 1.5, "temperature": 0.1, "weight_neg": 5000000.0, "weight_pos": 100.0, "weight_UWE": 10000.0, "neg_topk": 20}, "training": {"learning_rate": 0.001, "batch_size": 500, "num_epoch": 800}}

        model = topmost.CFDTM(
        vocab_size=dataset.vocab_size,
        train_time_wordfreq=dataset.train_time_wordfreq,
        num_times=dataset.num_times,
        pretrained_WE=dataset.pretrained_WE,
        num_topics=config["model"]["num_topics"],
        en_units=config["model"]["en1_units"],
        temperature=config["model"]["temperature"],
        beta_temp=config["model"]["beta_temp"],
        weight_neg=config["model"]["weight_neg"],
        weight_pos=config["model"]["weight_pos"],
        weight_UWE=config["model"]["weight_UWE"],
        neg_topk=config["model"]["neg_topk"],
        dropout=config["model"]["dropout"],
        embed_size=300
        )
  
        model = model.to(device)  
        trainer = topmost.DynamicTrainer(model, dataset, batch_size=config["training"]["batch_size"], learning_rate=config["training"]["learning_rate"], epochs=config["training"]["num_epoch"])
        top_words, train_theta = trainer.train()

        metrics_json, dynamic_TC, dynamic_TD, clustering, classification = eval(trainer, top_words)
        save_result_defensa(top_words, trainer, config, metrics_json, os.path.join(defensa_path, 'num_topics'))

        beta = model.get_beta()
        save_joblib(beta, os.path.join(defensa_path, 'num_topics', 'beta.joblib'))
        save_joblib(train_theta, os.path.join(defensa_path, 'num_topics', 'train_theta.joblib'))

## Modificacion: Clasificacion

In [19]:
        os.makedirs(os.path.join(defensa_path, 'metrics', 'model'), exist_ok=True)

        config = {"model": {"num_topics": 20, "en1_units": 50, "dropout": 0.1, "beta_temp": 1.5, "temperature": 0.1, "weight_neg": 5000000.0, "weight_pos": 100.0, "weight_UWE": 10000.0, "neg_topk": 20}, "training": {"learning_rate": 0.001, "batch_size": 500, "num_epoch": 800}}

        model = topmost.CFDTM(
        vocab_size=dataset.vocab_size,
        train_time_wordfreq=dataset.train_time_wordfreq,
        num_times=dataset.num_times,
        pretrained_WE=dataset.pretrained_WE,
        num_topics=config["model"]["num_topics"],
        en_units=config["model"]["en1_units"],
        temperature=config["model"]["temperature"],
        beta_temp=config["model"]["beta_temp"],
        weight_neg=config["model"]["weight_neg"],
        weight_pos=config["model"]["weight_pos"],
        weight_UWE=config["model"]["weight_UWE"],
        neg_topk=config["model"]["neg_topk"],
        dropout=config["model"]["dropout"],
        embed_size=300
        )

        model = model.to(device)  
        trainer = topmost.DynamicTrainer(model, dataset, batch_size=config["training"]["batch_size"], learning_rate=config["training"]["learning_rate"], epochs=config["training"]["num_epoch"])
        top_words, train_theta = trainer.train()

        metrics_json, _, _ = eval_clustering_cls(trainer, top_words)
        save_result_defensa(top_words, trainer, config, metrics_json, os.path.join(defensa_path, 'metrics', 'model'))

        beta = model.get_beta()
        save_joblib(beta, os.path.join(defensa_path, 'metrics', 'model', 'beta.joblib'))
        save_joblib(train_theta, os.path.join(defensa_path, 'metrics', 'model', 'train_theta.joblib'))
        save_joblib(model, os.path.join(defensa_path, 'metrics', 'model', 'model.joblib'))

100%|██████████| 800/800 [10:37<00:00,  1.25it/s]
22it [00:51,  2.33s/it]


dynamic TC list: [0.5677019265741741, 0.5473264402428053, 0.4923456246749159, 0.5432723375767776, 0.5042466019057775, 0.5522335065730417, 0.46608371362633494, 0.4993536214825299, 0.5213787349096782, 0.5174301246351893, 0.484013886743911, 0.5166642946714922, 0.5122213067585798, 0.45103387386478866, 0.4735774175769333, 0.4924615524516569, 0.49274238274259, 0.506652567639084, 0.5366530483354509, 0.4846558897407993, 0.49443885537141186, 0.586195160347525]
dynamic_TC:  0.5110310394747931


100%|██████████| 22/22 [00:00<00:00, 74.68it/s]


dynamic_TD:  0.9163636363636364


In [18]:
def eval_clustering_cls(trainer, top_words):
    # get theta (doc-topic distributions)
    train_theta, test_theta = trainer.export_theta()

    train_times = dataset.train_times.cpu().numpy()
    # compute topic coherence
    dynamic_TC = eva.dynamic_coherence(dataset.train_texts, train_times, dataset.vocab, top_words, verbose=True)
    print("dynamic_TC: ", dynamic_TC)

    # compute topic diversity
    dynamic_TD = eva.dynamic_diversity(top_words, dataset.train_bow.cpu().numpy(), train_times, dataset.vocab, verbose=False)
    print("dynamic_TD: ", dynamic_TD)
    
    json = {
        "dynamic_TC": dynamic_TC,
        "dynamic_TD": dynamic_TD,
    }
    
    save_json(json,  os.path.join(defensa_path, 'metrics', 'tc_td.joblib'))
    return json, dynamic_TC, dynamic_TD

eval_clustering_cls(trainer, top_words)

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/gensim/topic_coherence/direct_confirmation_measure.py:204: RuntimeWarning: divide by zero encountered in scalar divide
  m_lr_i = np.log(numerator / denominator)
/usr/local/lib/python3.10/dist-packages/gensim/topic_coherence/indirect_confirmation_measure.py:323: RuntimeWarning: invalid value encountered in scalar divide
  return cv1.T.dot(cv2)[0, 0] / (_magnitude(cv1) * _magnitude(cv2))
22it [00:47,  2.16s/it]


dynamic TC list: [0.5474639207609946, 0.5485055440911806, 0.538093315991761, 0.5595035827013972, 0.5571586879373494, 0.5710942423312598, 0.49727653490284063, 0.5057157578628461, 0.5364983797496669, 0.5380682684098397, 0.5307859345809531, 0.5361257264203786, 0.5426859438886168, 0.5097462204554137, 0.5385772852900975, 0.49183257804142605, 0.5622213206511859, 0.5586238061437698, 0.5825828057721377, 0.5403927587880044, 0.5232952103670785, 0.5312220862599667]
dynamic_TC:  0.5385213596090075


100%|██████████| 22/22 [00:00<00:00, 64.48it/s]

dynamic_TD:  0.8746969696969699


({'dynamic_TC': 0.5385213596090075, 'dynamic_TD': 0.8746969696969699},
 0.5385213596090075,
 0.8746969696969699)

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, f1_score

def train_and_evaluate(X_train, X_test, y_train, y_test, model, params):
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train) 
    X_test = scaler.transform(X_test)
    
    # for name, (model, params) in models.items():
    if params:  # Naive Bayes no requiere hiperparámetros
        grid_search = GridSearchCV(model, params, scoring='accuracy', cv=5)
        grid_search.fit(X_train, y_train)
        best_estimator = grid_search.best_estimator_
    else:
        model.fit(X_train, y_train)
        best_estimator = model
        
    y_pred = best_estimator.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"Params: {grid_search.best_params_}")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("Reporte de clasificación:")
    print(classification_report(y_test, y_pred))
        
    return {
        "params": grid_search.best_params_,
        "metrics": {
            "acc": acc,
            "macro-f1": f1},
        "classification_report": classification_report(y_test, y_pred)
        }

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

def eval_cls(X_train, X_test, y_train, y_test):
    n_train, n_features = X_train.shape
    
    rf_params = {"n_estimators": [n_train // 10, n_train // 5, n_train // 2]}
    rf = train_and_evaluate(X_train, X_test, y_train, y_test, RandomForestClassifier(), rf_params)
    print("Random Forest: ", rf)
    
    knn_params = {"n_neighbors": [int(np.sqrt(n_train)) // 2, int(np.sqrt(n_train)), int(np.sqrt(n_train)) * 2]}
    knn = train_and_evaluate(X_train, X_test, y_train, y_test, KNeighborsClassifier(), knn_params)
    print("KNN: ", knn)

    svm_params = {"kernel": ['linear', 'rbf']} #, "C": [0.1, 1, n_features]
    svm = train_and_evaluate(X_train, X_test, y_train, y_test, SVC(), svm_params)
    print("SVM: ", svm)
    
    return {
        "rf": rf,
        "knn": knn,
        "svm": svm
    }

In [15]:
train_theta, test_theta = trainer.export_theta()
train_times = dataset.train_times.cpu().numpy()

classification = eval_cls(train_theta, test_theta, dataset.train_labels, dataset.test_labels)
save_json(classification, os.path.join(defensa_path, 'metrics', 'classification.json'))

Params: {'n_estimators': 241}
Accuracy: 0.4726
F1 Score: 0.4288
Reporte de clasificación:
              precision    recall  f1-score   support

           0       1.00      0.04      0.08        23
           1       1.00      0.13      0.23        39
           2       0.43      0.87      0.58       145
           3       0.58      0.67      0.62        64
           4       0.49      0.34      0.40        56
           5       0.56      0.12      0.19        43
           6       0.67      0.21      0.32        38
           7       1.00      0.09      0.17        22
           8       0.53      0.45      0.48        65
           9       1.00      0.14      0.25         7
          10       0.37      0.52      0.43       106
          11       0.00      0.00      0.00         9
          12       1.00      0.14      0.25         7
          13       0.84      0.47      0.60        34

    accuracy                           0.47       658
   macro avg       0.68      0.30      0.33 

## Clustering

In [20]:
from scipy.stats import mode
def purity_score(y_true, y_pred):
    labels = np.unique(y_pred)
    majority_sum = 0
    for label in labels:
        mask = y_pred == label
        majority_class = mode(y_true[mask])[0][0]
        majority_sum += np.sum(y_true[mask] == majority_class)
    return majority_sum / len(y_true)

In [21]:
from sklearn import metrics
import numpy as np

def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix)

In [26]:
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import normalized_mutual_info_score
from tqdm import tqdm

def best_config(X_test, y_test, model, model_name, params):
    best_local_model = None
    best_local_score = -1
    best_local_nmi = -1
    best_local_purity = -1
    best_local_params = None
        
    for param_set in tqdm(ParameterGrid(params), desc=model_name):
        model.set_params(**param_set)
        labels = model.fit_predict(X_test)
        if len(set(labels)) > 1:  # Evitar errores con un solo cluster
            purity = purity_score(y_test, labels)
            nmi = normalized_mutual_info_score(y_test, labels)
            score = (purity + nmi) / 2  # Promedio de ambas métricas
                
            if score > best_local_score:
                best_local_score = score
                best_local_model = model
                best_local_params = param_set
                best_local_nmi = nmi
                best_local_purity = purity

    print(f"Params: {best_local_params}")
    print(f"Purity: {purity:.4f}")
    print(f"NMI: {nmi:.4f}")
    
    return {
        "params": best_local_params,
        "metrics": {
            "purity": purity,
            "nmi": nmi},
        # "classification_report": classification_report(y_test, y_pred)
        }

def clustermax_probability(theta, labels):
    preds = np.argmax(theta, axis=1)
    purity = purity_score(labels, preds)
    nmi = normalized_mutual_info_score(labels, preds)
    
    print("Max Probability")
    print(f"Purity: {purity:.4f}")
    print(f"NMI: {nmi:.4f}")
    
    return {
        "purity": purity,
        "nmi": nmi
    }

In [27]:
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
import numpy as np

def eval_clustering(test_theta, test_labels):
    kmeans_params = {"n_clusters": [5, 10, 14, 20], "init": ['k-means++', 'random'], "max_iter": [300, 500]}
    kmeans = best_config(test_theta, test_labels, KMeans(), 'K-Means', kmeans_params)

    
    dbscan_params = {"eps": [0.1, 0.2, 0.5], "min_samples": [3, 5, 10], "metric": ['euclidean', 'cosine']}
    dbscan = best_config(test_theta, test_labels, DBSCAN(), 'DBSCAN', dbscan_params)

    hierarchical_params = {"n_clusters": [5, 10, 14, 20], "affinity": ['euclidean', 'cosine'], "linkage": ['complete', 'average']}
    hierarchical = best_config(test_theta, test_labels, AgglomerativeClustering(), 'Agglomerative Clustering', hierarchical_params)

    max_probability = clustermax_probability(test_theta, test_labels)

    return {
        "kmeans": kmeans,
        "dbscan": dbscan,
        "hierarchical": hierarchical,
        "max_probability": max_probability
    }

train_theta, test_theta = trainer.export_theta()
train_times = dataset.train_times.cpu().numpy()
clustering = eval_clustering(test_theta, dataset.test_labels)
save_json(clustering, os.path.join(defensa_path, 'metrics', 'clustering.json'))

K-Means: 100%|██████████| 16/16 [00:00<00:00, 22.74it/s]


Params: {'init': 'k-means++', 'max_iter': 300, 'n_clusters': 20}
Purity: 0.4210
NMI: 0.2979


DBSCAN: 100%|██████████| 18/18 [00:00<00:00, 130.87it/s]


Params: {'eps': 0.1, 'metric': 'euclidean', 'min_samples': 3}
Purity: 0.2219
NMI: 0.0030


Agglomerative Clustering: 100%|██████████| 16/16 [00:00<00:00, 91.39it/s]

Params: {'affinity': 'cosine', 'linkage': 'average', 'n_clusters': 20}
Purity: 0.4149
NMI: 0.2992
Max Probability
Purity: 0.4164
NMI: 0.2973


In [1]:
print('hello')

hello
